In [114]:
import os
from dotenv import load_dotenv
from twarc import Twarc2
from twarc_csv import DataFrameConverter
import urllib
from neo4j import GraphDatabase

from tweet_processing import pull_tweets, get_user_following

load_dotenv()

neo4jUrl = os.environ.get('NEO4J_URI')
neo4jUser = os.environ.get('NEO4J_USER',"neo4j")
neo4jPass = os.environ.get('NEO4J_PASSWORD',"test")
bearerToken = os.environ.get('TWITTER_BEARER',"")


twarc_client = Twarc2(
    consumer_key=os.environ["consumer_key"],
    consumer_secret=os.environ["consumer_secret"],
    access_token=os.environ["access_token"],
    access_token_secret=os.environ["access_token_secret"],
)

driver = GraphDatabase.driver(neo4jUrl, auth=(neo4jUser, neo4jPass))

2022-09-08 12:33:12.780 INFO    twarc: creating user auth client


In [35]:
# _, df_tweets, df_ref_tweets = pull_tweets(twarc_client, username="nicktorba", extract_features=True, max_tweets=100) #, start_time=None, end_time=None):


In [115]:
df = get_user_following(twarc_client, "nicktorba")
df.head()

2022-09-08 12:33:32.333 INFO    twarc: getting ('https://api.twitter.com/2/users/by',) {'params': {'tweet.fields': 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,text,possibly_sensitive,referenced_tweets,reply_settings,source,withheld', 'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld', 'usernames': 'nicktorba'}}


fetching accounts followed by nicktorba


2022-09-08 12:33:32.628 INFO    twarc: getting ('https://api.twitter.com/2/users/803693608419422209/following',) {'params': {'tweet.fields': 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,text,possibly_sensitive,referenced_tweets,reply_settings,source,withheld', 'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld', 'max_results': 1000}}


,id,created_at,username,name,description,entities.description.cashtags,entities.description.hashtags,entities.description.mentions,entities.description.urls,entities.url.urls,...,url,verified,withheld.scope,withheld.copyright,withheld.country_codes,__twarc.retrieved_at,__twarc.url,__twarc.version,,referencer.username
0,19904963,2009-02-02T12:56:46.000Z,jappleby,Jack Appleby ☕️,"Creator of Future Social, @MorningBrew's newsl...",NaN,NaN,"[{""start"": 26, ""end"": 38, ""username"": ""Morning...",NaN,"[{""start"": 0, ""end"": 23, ""url"": ""https://t.co/...",...,https://t.co/rypBxqDRLy,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nicktorba
1,1155847830809899009,2019-07-29T14:29:32.000Z,akbirthko,akbirthko,,NaN,NaN,NaN,NaN,NaN,...,,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nicktorba
2,732980797985148928,2016-05-18T17:06:55.000Z,empathy2000,cam,"“sudden glory”, as Hobbes says of laughter",NaN,NaN,NaN,NaN,"[{""start"": 0, ""end"": 23, ""url"": ""https://t.co/...",...,https://t.co/WZHjR4Vhvw,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nicktorba
3,1432885307872096256,2021-09-01T01:58:22.000Z,spewbound,fafner,talking dog on an intergalactic journey. ratbo...,NaN,NaN,NaN,NaN,NaN,...,,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nicktorba
4,422135704,2011-11-26T21:22:36.000Z,OptimistsInc,Jess Sorrell,"Postdoc @CIS_Penn, PhD @ucsd_cse. ML theory, r...",NaN,NaN,"[{""start"": 8, ""end"": 17, ""username"": ""CIS_Penn...","[{""start"": 130, ""end"": 153, ""url"": ""https://t....",NaN,...,,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nicktorba


In [119]:
df[author_fields].columns

Index(['id', 'created_at', 'username', 'name', 'description', 'location',
       'pinned_tweet_id', 'profile_image_url', 'protected',
       'public_metrics.followers_count', 'public_metrics.following_count',
       'public_metrics.listed_count', 'public_metrics.tweet_count', 'url',
       'verified'],
      dtype='object')

In [112]:
author_fields = author_fields = ['id', 'created_at', 'username', 'name', 
'description', 
# 'entities.description.cashtags', 'entities.description.hashtags', 'entities.description.mentions', 'entities.description.urls', 'entities.url.urls', 
'location', 'pinned_tweet_id', 'profile_image_url', 'protected', 
'public_metrics.followers_count', 'public_metrics.following_count', 'public_metrics.listed_count', 'public_metrics.tweet_count', 'url', 'verified', 
# 'withheld.scope', 'withheld.copyright', 'withheld.country_codes'
]
len(author_fields)

15

In [113]:
# df_tweets.columns

In [33]:
# import json
# json.loads(df_tweets[df_tweets["entities.mentions"].notna()].iloc[0]["entities.mentions"])

In [111]:
# df["entities.mentions"].map(lambda x: json.loads(x) if isinstance(x, str) else x)

mentions = []
for i in df["entities.mentions"].tolist():
    if isinstance(i, str):
        i_mentions = json.loads(i)
        [i.pop("end") if "end" in i else i for i in i_mentions]
        [i.pop("start") if "start" in i else i for i in i_mentions]
        # j = DataFrameConverter("users").process([i_mention])
        mentions.append(DataFrameConverter("users").process(i_mentions).to_dict("records"))
    else:
        mentions.append([])
        
# flat_mentions = []
# for i in mentions:
#     if len(i)>0:
#         flat_mentions.append(DataFrameConverter("users").process(i).to_dict("records"))
#     else: 
#         flat_mentions.append(i)

mentions

[[],
 [],
 [{'id': '495430242',
   'created_at': '2012-02-17T23:34:50.000Z',
   'username': 'IgorBrigadir',
   'name': 'Igor Brigadir 🇺🇦',
   'description': 'CTO @recsyslabs. Adjunct Research Fellow @ucddublin. PhD @insight_centre. Recommender Systems, Information Retrieval, Data Science, NLProc, ML, AI. he/him',
   'entities.description.cashtags': nan,
   'entities.description.hashtags': nan,
   'entities.description.mentions': '[{"start": 4, "end": 15, "username": "recsyslabs"}, {"start": 41, "end": 51, "username": "ucddublin"}, {"start": 57, "end": 72, "username": "insight_centre"}]',
   'entities.description.urls': nan,
   'entities.url.urls': '[{"start": 0, "end": 23, "url": "https://t.co/rT5vLfqHOC", "expanded_url": "https://ukrainewar.carrd.co/", "display_url": "ukrainewar.carrd.co"}]',
   'location': 'Ireland',
   'pinned_tweet_id': '1496861088519897090',
   'profile_image_url': 'https://pbs.twimg.com/profile_images/2538946114/xiveugt78rc97y1dasxf_normal.jpeg',
   'protected': 

In [102]:
mentions[2]

[{'username': 'IgorBrigadir',
  'id': '495430242',
  'public_metrics': {'followers_count': 3360,
   'following_count': 5007,
   'tweet_count': 29477,
   'listed_count': 185},
  'protected': False,
  'url': 'https://t.co/rT5vLfqHOC',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/2538946114/xiveugt78rc97y1dasxf_normal.jpeg',
  'verified': False,
  'pinned_tweet_id': '1496861088519897090',
  'location': 'Ireland',
  'name': 'Igor Brigadir 🇺🇦',
  'created_at': '2012-02-17T23:34:50.000Z',
  'entities': {'url': {'urls': [{'start': 0,
      'end': 23,
      'url': 'https://t.co/rT5vLfqHOC',
      'expanded_url': 'https://ukrainewar.carrd.co/',
      'display_url': 'ukrainewar.carrd.co'}]},
   'description': {'mentions': [{'start': 4,
      'end': 15,
      'username': 'recsyslabs'},
     {'start': 41, 'end': 51, 'username': 'ucddublin'},
     {'start': 57, 'end': 72, 'username': 'insight_centre'}]}},
  'description': 'CTO @recsyslabs. Adjunct Research Fellow @ucddublin. PhD @in

set()

In [106]:
DataFrameConverter("users").process(mentions[2]).to_dict("records")

[{'id': '495430242',
  'created_at': '2012-02-17T23:34:50.000Z',
  'username': 'IgorBrigadir',
  'name': 'Igor Brigadir 🇺🇦',
  'description': 'CTO @recsyslabs. Adjunct Research Fellow @ucddublin. PhD @insight_centre. Recommender Systems, Information Retrieval, Data Science, NLProc, ML, AI. he/him',
  'entities.description.cashtags': nan,
  'entities.description.hashtags': nan,
  'entities.description.mentions': '[{"start": 4, "end": 15, "username": "recsyslabs"}, {"start": 41, "end": 51, "username": "ucddublin"}, {"start": 57, "end": 72, "username": "insight_centre"}]',
  'entities.description.urls': nan,
  'entities.url.urls': '[{"start": 0, "end": 23, "url": "https://t.co/rT5vLfqHOC", "expanded_url": "https://ukrainewar.carrd.co/", "display_url": "ukrainewar.carrd.co"}]',
  'location': 'Ireland',
  'pinned_tweet_id': '1496861088519897090',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/2538946114/xiveugt78rc97y1dasxf_normal.jpeg',
  'protected': False,
  'public_metrics

In [64]:
res = twarc_client.timeline("803693608419422209",  max_results=5)
d = next(res)

2022-09-08 09:04:21.660 INFO    twarc: getting ('https://api.twitter.com/2/users/803693608419422209/tweets',) {'params': {'expansions': 'author_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id,entities.mentions.username,attachments.poll_ids,attachments.media_keys,geo.place_id', 'tweet.fields': 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,text,possibly_sensitive,referenced_tweets,reply_settings,source,withheld', 'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld', 'media.fields': 'alt_text,duration_ms,height,media_key,preview_image_url,type,url,width,public_metrics', 'poll.fields': 'duration_minutes,end_datetime,id,options,voting_status', 'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type', 'start_time': None, 'end_time': None, 'max_results': 

In [76]:
d["includes"]["users"][0].keys()

dict_keys(['created_at', 'public_metrics', 'verified', 'location', 'entities', 'pinned_tweet_id', 'profile_image_url', 'protected', 'description', 'id', 'url', 'username', 'name'])

In [72]:
DataFrameConverter("users").process(d["includes"]["users"])

,id,created_at,username,name,description,entities.description.cashtags,entities.description.hashtags,entities.description.mentions,entities.description.urls,entities.url.urls,...,public_metrics.tweet_count,url,verified,withheld.scope,withheld.copyright,withheld.country_codes,__twarc.retrieved_at,__twarc.url,__twarc.version,
0,803693608419422209,2016-11-29T20:14:22.000Z,nicktorba,Nick Torba,"everywhere I go, there I am | building @Tweets...",NaN,NaN,"[{""start"": 39, ""end"": 52, ""username"": ""Tweetsc...","[{""start"": 81, ""end"": 104, ""url"": ""https://t.c...","[{""start"": 0, ""end"": 23, ""url"": ""https://t.co/...",...,4118,https://t.co/C8yIPStHx3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18280363,2008-12-21T06:17:31.000Z,TylerAlterman,Tyler Alterman,Breathe in the evening sun. Breathe out the do...,NaN,NaN,NaN,NaN,"[{""start"": 0, ""end"": 23, ""url"": ""https://t.co/...",...,14462,https://t.co/FHr17Be1i1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51882249,2009-06-28T23:31:11.000Z,MeetKevon,Kevon Cheung 🥦,I help you grow a community around your work b...,NaN,NaN,NaN,"[{""start"": 103, ""end"": 126, ""url"": ""https://t....","[{""start"": 0, ""end"": 23, ""url"": ""https://t.co/...",...,24180,https://t.co/f3ZdRcUFLp,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1320858124971106305,2020-10-26T22:41:48.000Z,ollyrobot,o||y,ó na coillte,NaN,NaN,NaN,NaN,NaN,...,48903,,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
help(ensure_flattened)

Help on function ensure_flattened in module twarc.expansions:

ensure_flattened(data)
    Will ensure that the supplied data is "flattened". The input data can be a
    response from the Twitter API, a list of tweet dictionaries, or a single tweet
    dictionary. It will always return a list of tweet dictionaries. A ValueError
    will be thrown if the supplied data is not recognizable or it cannot be
    flattened.
    
    ensure_flattened is designed for use in twarc plugins and other tweet
    processing applications that want to operate on a stream of tweets, and
    examine included entities like users and tweets without hunting and
    pecking in the response data.



In [42]:
import pandas as pd
df = pd.read_csv("localdffortest.csv")
json.loads(df[df["entities.mentions"].notna()].iloc[0]["entities.mentions"])

[{'start': 84,
  'end': 97,
  'username': 'IgorBrigadir',
  'id': '495430242',
  'public_metrics': {'followers_count': 3360,
   'following_count': 5007,
   'tweet_count': 29477,
   'listed_count': 185},
  'protected': False,
  'url': 'https://t.co/rT5vLfqHOC',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/2538946114/xiveugt78rc97y1dasxf_normal.jpeg',
  'verified': False,
  'pinned_tweet_id': '1496861088519897090',
  'location': 'Ireland',
  'name': 'Igor Brigadir 🇺🇦',
  'created_at': '2012-02-17T23:34:50.000Z',
  'entities': {'url': {'urls': [{'start': 0,
      'end': 23,
      'url': 'https://t.co/rT5vLfqHOC',
      'expanded_url': 'https://ukrainewar.carrd.co/',
      'display_url': 'ukrainewar.carrd.co'}]},
   'description': {'mentions': [{'start': 4,
      'end': 15,
      'username': 'recsyslabs'},
     {'start': 41, 'end': 51, 'username': 'ucddublin'},
     {'start': 57, 'end': 72, 'username': 'insight_centre'}]}},
  'description': 'CTO @recsyslabs. Adjunct Researc

In [13]:
df_tweets[df_tweets["entities.mentions"].notna()].iloc[1]["entities.mentions.usernames"]

'RhysLindmark, sashachapin'

In [19]:
df_tweets["created_at"] = df_tweets["created_at"].map(lambda x: x.isoformat())
# df_tweets["created_at.hour"] = df_tweets["created_at.hour"].map(lambda x: x.isoformat())


AttributeError: 'str' object has no attribute 'isoformat'

In [31]:
author_cols = [i for i in df_tweets.columns if i.startswith("author.")]
author_cols

author_records = df_tweets[author_cols]

author_cols_rename = {}
for i in author_cols:
    ind = i.find(".")
    author_cols_rename[i] = i[ind+1:]

author_records.rename(columns=author_cols_rename, inplace=True)

/tmp/ipykernel_268005/596188096.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_records.rename(columns=author_cols_rename, inplace=True)


In [26]:
"adsf.adf.adf".find(".")

4

In [27]:
"adsf.adf.adf"[4:]

'.adf.adf'

In [23]:
# df_tweets.to_dict("records")